In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 300
l = 10
num_filter = 20

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
# Normalize input Data, value in between 0 and 1
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [6]:
# Dense Block
def add_denseblock(input):
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(relu)
        Dropout_Layer = Dropout(0.2)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Dropout_Layer])
        temp = concat
        
    return temp

In [7]:
def add_transition(input):
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(num_filter, (1,1), use_bias=False ,padding='same')(relu)
    Dropout_Layer = Dropout(0.2)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Dropout_Layer)
    
    return avg

In [8]:
def output_layer(input):
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D)
First_Transition = add_transition(First_Block)

Second_Block = add_denseblock(First_Transition)
Second_Transition = add_transition(Second_Block)

Third_Block = add_denseblock(Second_Transition)
Third_Transition = add_transition(Third_Block)

Last_Block = add_denseblock(Third_Transition)
output = output_layer(Last_Block)

'''
Epoch 50/50
loss: 0.0695 - acc: 0.9756 - val_loss: 1.1741 - val_acc: 0.7874
'''

'\nEpoch 50/50\nloss: 0.0695 - acc: 0.9756 - val_loss: 1.1741 - val_acc: 0.7874\n'

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/300
50000/50000 [==============================] - 235s 5ms/step - loss: 1.4451 - acc: 0.4676 - val_loss: 2.8243 - val_acc: 0.2559
Epoch 2/300
50000/50000 [==============================] - 210s 4ms/step - loss: 1.0188 - acc: 0.6333 - val_loss: 1.1735 - val_acc: 0.6198
Epoch 3/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.8296 - acc: 0.7039 - val_loss: 1.0868 - val_acc: 0.6628
Epoch 4/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.7309 - acc: 0.7385 - val_loss: 1.9132 - val_acc: 0.5308
Epoch 5/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.6572 - acc: 0.7678 - val_loss: 0.8317 - val_acc: 0.7314
Epoch 6/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.5990 - acc: 0.7871 - val_loss: 1.5741 - val_acc: 0.5828
Epoch 7/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.5493 - acc: 0.8058 - val_loss: 0.

50000/50000 [==============================] - 210s 4ms/step - loss: 0.0961 - acc: 0.9654 - val_loss: 0.7760 - val_acc: 0.8436
Epoch 60/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0939 - acc: 0.9668 - val_loss: 0.9277 - val_acc: 0.8198
Epoch 61/300
50000/50000 [==============================] - 208s 4ms/step - loss: 0.0905 - acc: 0.9673 - val_loss: 0.6941 - val_acc: 0.8579
Epoch 62/300
50000/50000 [==============================] - 208s 4ms/step - loss: 0.0917 - acc: 0.9670 - val_loss: 0.7047 - val_acc: 0.8551
Epoch 63/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0902 - acc: 0.9676 - val_loss: 0.5544 - val_acc: 0.8799
Epoch 64/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0912 - acc: 0.9666 - val_loss: 0.6549 - val_acc: 0.8688
Epoch 65/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0860 - acc: 0.9696 - val_loss: 0.6635 - val_acc: 0.8660
Epoch 66/300
50000/50000 [=======

50000/50000 [==============================] - 209s 4ms/step - loss: 0.0481 - acc: 0.9831 - val_loss: 0.7432 - val_acc: 0.8661
Epoch 118/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.0507 - acc: 0.9817 - val_loss: 0.6430 - val_acc: 0.8828
Epoch 119/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0459 - acc: 0.9835 - val_loss: 0.6350 - val_acc: 0.8865
Epoch 120/300
50000/50000 [==============================] - 212s 4ms/step - loss: 0.0456 - acc: 0.9833 - val_loss: 0.7789 - val_acc: 0.8620
Epoch 121/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0498 - acc: 0.9830 - val_loss: 0.6552 - val_acc: 0.8818
Epoch 122/300
50000/50000 [==============================] - 212s 4ms/step - loss: 0.0462 - acc: 0.9838 - val_loss: 0.7062 - val_acc: 0.8764
Epoch 123/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.0454 - acc: 0.9840 - val_loss: 0.7320 - val_acc: 0.8735
Epoch 124/300
50000/50000 [

50000/50000 [==============================] - 212s 4ms/step - loss: 0.0329 - acc: 0.9886 - val_loss: 0.6633 - val_acc: 0.8855
Epoch 176/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0342 - acc: 0.9879 - val_loss: 0.6495 - val_acc: 0.8879
Epoch 177/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.0371 - acc: 0.9875 - val_loss: 0.7856 - val_acc: 0.8623
Epoch 178/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0308 - acc: 0.9895 - val_loss: 0.6793 - val_acc: 0.8812
Epoch 179/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0367 - acc: 0.9868 - val_loss: 0.7362 - val_acc: 0.8757
Epoch 180/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.0309 - acc: 0.9893 - val_loss: 0.6876 - val_acc: 0.8810
Epoch 181/300
50000/50000 [==============================] - 211s 4ms/step - loss: 0.0316 - acc: 0.9896 - val_loss: 0.6930 - val_acc: 0.8829
Epoch 182/300
50000/50000 [

50000/50000 [==============================] - 210s 4ms/step - loss: 0.0255 - acc: 0.9910 - val_loss: 0.6916 - val_acc: 0.8879
Epoch 234/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0233 - acc: 0.9921 - val_loss: 0.6573 - val_acc: 0.8927
Epoch 235/300
50000/50000 [==============================] - 208s 4ms/step - loss: 0.0270 - acc: 0.9909 - val_loss: 0.6687 - val_acc: 0.8901
Epoch 236/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0256 - acc: 0.9910 - val_loss: 0.6249 - val_acc: 0.8959
Epoch 237/300
50000/50000 [==============================] - 208s 4ms/step - loss: 0.0256 - acc: 0.9911 - val_loss: 0.6467 - val_acc: 0.8925
Epoch 238/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0260 - acc: 0.9910 - val_loss: 0.6369 - val_acc: 0.8946
Epoch 239/300
50000/50000 [==============================] - 210s 4ms/step - loss: 0.0253 - acc: 0.9910 - val_loss: 0.6659 - val_acc: 0.8895
Epoch 240/300
50000/50000 [

50000/50000 [==============================] - 209s 4ms/step - loss: 0.0250 - acc: 0.9918 - val_loss: 0.6293 - val_acc: 0.8974
Epoch 292/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0209 - acc: 0.9929 - val_loss: 0.6736 - val_acc: 0.8940
Epoch 293/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0229 - acc: 0.9916 - val_loss: 0.6580 - val_acc: 0.8921
Epoch 294/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0235 - acc: 0.9924 - val_loss: 0.6371 - val_acc: 0.8920
Epoch 295/300
50000/50000 [==============================] - 208s 4ms/step - loss: 0.0204 - acc: 0.9929 - val_loss: 0.6792 - val_acc: 0.8907
Epoch 296/300
50000/50000 [==============================] - 206s 4ms/step - loss: 0.0189 - acc: 0.9936 - val_loss: 0.7049 - val_acc: 0.8907
Epoch 297/300
50000/50000 [==============================] - 209s 4ms/step - loss: 0.0238 - acc: 0.9916 - val_loss: 0.6880 - val_acc: 0.8887
Epoch 298/300
50000/50000 [

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

'''
Test accuracy: 0.8972
'''

Test accuracy: 0.8972


Epoch 298/300
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0024 - acc: 0.9993 - val_loss: 1.0791 - val_acc: 0.8551
Epoch 299/300
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0027 - acc: 0.9991 - val_loss: 1.1468 - val_acc: 0.8472
Epoch 300/300
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0050 - acc: 0.9983 - val_loss: 1.2931 - val_acc: 0.8339
Test loss: 1.2930997592
Test accuracy: 0.8339